In [3]:
import json
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [22]:
# Load the JSON file
with open('model_outputs_Meta-Llama-3-8B-Instruct_5shots.json', 'r') as file:
    data = json.load(file)

# Convert the data to a DataFrame
df = pd.DataFrame(data)
df.head()

,question_id,question,options,answer,answer_index,cot_content,category,src,pred,generated_text
0,70,"Typical advertising regulatory bodies suggest,...","[Safe practices, Fear, Jealousy, Trivial, Unsa...",I,8,,business,ori_mmlu-business_ethics,I,We refer to Wikipedia articles on advertising...
1,71,Managers are entrusted to run the company in t...,"[Shareholders, Diligence, Self-interest, Share...",F,5,,business,ori_mmlu-business_ethics,B,We refer to Wikipedia articles on business et...
2,72,There are two main issues associated with ____...,"[Down, Autonomy, Remuneration, Benefit, Down, ...",J,9,,business,ori_mmlu-business_ethics,A,We refer to Wikipedia articles on business et...
3,73,_______ locate morality beyond the sphere of r...,"[Ethical egoism, Ethics of duty, Postmodern et...",C,2,,business,ori_mmlu-business_ethics,H,We refer to Wikipedia articles on ethics for ...
4,74,Some of key differences between Islamic finan...,"[Interest, Certain, Assured, Both tangible and...",G,6,,business,ori_mmlu-business_ethics,A,We refer to Wikipedia articles on Islamic fin...


In [11]:


# Load the embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Compute embeddings for existing questions
df['embedding'] = df['question'].apply(lambda x: model.encode(x))
# Create a new DataFrame with only 'question' and 'embedding' columns
df_questions_embeddings = df[['question', 'embedding']]

df_questions_embeddings.to_pickle('questions_embeddings.pkl')

# Save the new DataFrame to a CSV file
# df_questions_embeddings.to_csv('questions_embeddings.csv', index=False)

In [21]:
df_similarity = pd.read_pickle('questions_embeddings.pkl')

# Prompt user for input question
model = SentenceTransformer('all-MiniLM-L6-v2')
input_question = input("Enter your question: ")

# Compute embedding for input question
input_embedding = model.encode(input_question)

# Compute similarities
df_similarity['similarity'] = df_similarity['embedding'].apply(lambda x: cosine_similarity([input_embedding], [x])[0][0])

# Find the top 20 most similar questions
top20 = df_similarity.nlargest(20, 'similarity')

# Load the model performance data
df_performance = pd.read_pickle('model_performance.pkl')

# Filter the performance data for the top 20 questions
top20_questions = top20['question'].values
df_top20_performance = df_performance[df_performance['question'].isin(top20_questions)]

# Calculate the mean performance for each model
model_performance = df_top20_performance.drop(columns=['question']).mean().sort_values(ascending=False)

# Select the top 3 models
top3_models = model_performance.head(3)

# Display the top 3 models
print("Top 3 models for the input question:")
print(top3_models)

Top 3 models for the input question:
arx                  0.85
Meta-Llama-3         0.85
gpt-4o-2024-08-06    0.80
dtype: float64


In [6]:
# Load the CSV file
df_performance = pd.read_csv('model_performance.csv')

# Display the head of the DataFrame
df_performance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12515 entries, 0 to 12514
Data columns (total 43 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   question                    12515 non-null  object
 1   Llama-2-13b-hf              12515 non-null  int64 
 2   Llama-2-70b-hf              12515 non-null  int64 
 3   Llama-2-7b-hf               12515 non-null  int64 
 4   Meta-Llama-3                12515 non-null  int64 
 5   Meta-Llama-3-70B            12515 non-null  int64 
 6   Meta-Llama-3-70B-Instruct   12515 non-null  int64 
 7   Meta-Llama-3-8B             12515 non-null  int64 
 8   Meta-Llama-3-8B-Instruct    12515 non-null  int64 
 9   Mistral-7B-Instruct-v0.1    12515 non-null  int64 
 10  Mistral-7B-Instruct-v0.2    12515 non-null  int64 
 11  Mistral-7B-v0.1             12515 non-null  int64 
 12  Mistral-7B-v0.2-hf          12515 non-null  int64 
 13  Mixtral-8x7B-Instruct-v0.1  12515 non-null  in